In [12]:
print("Hello World")

Hello World


In [13]:
import os
%pwd

'd:\\Jeevan\\VSCode\\Text_summarization\\text_summarization'

In [14]:
os.chdir("../")

In [15]:
%pwd

'd:\\Jeevan\\VSCode\\Text_summarization'

In [16]:
from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml,create_dirs

In [17]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir:Path
    data_path:Path
    tokenizer_name:Path

In [18]:
class ConfigurationManager:
    def __init__(self,config_filepath=CONFIG_FILE_PATH,params_filepath=PARAMS_FILE_PATH):
        self.config=read_yaml(config_filepath)
        self.params=read_yaml(params_filepath)
        create_dirs([self.config.artifacts_root])
    def get_data_transformation_config(self)->DataTransformationConfig:
        config=self.config.data_transformation
        create_dirs([config.root_dir])

        data_transformation_config=DataTransformationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            tokenizer_name=config.tokenizer_name
        )
        return data_transformation_config

In [19]:
# !pip install transformers[sentencepiece] datasets sacrebleu rouge_score py7zr -q

In [20]:
from transformers import AutoTokenizer
from textSummarizer.logging import logger
from datasets import load_from_disk,load_dataset

In [21]:
class DataTransformation:
    def __init__(self,config:DataTransformationConfig):
        self.config=config
        self.tokenizer = AutoTokenizer.from_pretrained(config.tokenizer_name)
    
    def convert_examples_to_features(self,example_batch):
        input_encodings = self.tokenizer(example_batch['dialogue'] , max_length = 1024, truncation = True,padding="max_length" )

        with self.tokenizer.as_target_tokenizer():
            target_encodings = self.tokenizer(example_batch['summary'], max_length = 128, truncation = True,padding="max_length")

        return {
            'input_ids' : input_encodings['input_ids'],
            'attention_mask': input_encodings['attention_mask'],
            'labels': target_encodings['input_ids']
        }

    def convert(self):
        dataset_samsum=load_from_disk(self.config.data_path)
        dataset_samsum_pt = dataset_samsum.map(self.convert_examples_to_features, batched = True)
        dataset_samsum_pt.save_to_disk(os.path.join(self.config.root_dir,"samsum_dataset"))
        print(dataset_samsum_pt['train'][0])

In [ ]:
try:
    config=ConfigurationManager()
    data_transformtaion_config=config.get_data_transformation_config()
    data_transformtaion=DataTransformation(config=data_transformtaion_config)
    data_transformtaion.convert()
except ValueError as e:
    raise e